<a href="https://colab.research.google.com/github/ai4ai-lab/Reward-Shaping/blob/main/Lifegate_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import os
from google.colab import drive
import pickle
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from scipy.optimize import minimize
import random
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import torch

# deadend dependencies

In [2]:
!git clone https://github.com/microsoft/med-deadend.git


Cloning into 'med-deadend'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 130 (delta 52), reused 77 (delta 22), pack-reused 0
Receiving objects: 100% (130/130), 395.48 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# shaping dependencies

In [3]:
!git clone https://github.com/ajagota7/Shaping.git

Cloning into 'Shaping'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 101 (delta 49), reused 74 (delta 28), pack-reused 0
Receiving objects: 100% (101/101), 11.74 MiB | 19.42 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [4]:
# %cd /content/Shaping

In [5]:
# !git pull origin main

In [6]:
# cd /content/

In [7]:
# %cd /content/Shaping

import zipfile

with zipfile.ZipFile('/content/Shaping/lifegate_1.zip', 'r') as zip_ref:
    # zip_ref.extractall('/content/med-deadend/lifegate/results/lifegate_1')
    zip_ref.extractall('/content/Shaping/')

In [8]:
%cd /content/med-deadend/lifegate


# results_dir = 'results/lifegate_1/'
results_dir = '/content/Shaping/'
# Load the Q tables from the primary learning agent, Q_D and Q_R value functions
with open(results_dir+'tabular_qnet.pkl', 'rb') as fq:
    ai = pickle.load(fq)

with open(results_dir+'tabular_qd.pkl', 'rb') as fd:
    ai_d = pickle.load(fd)

with open(results_dir+'tabular_qr.pkl', 'rb') as fr:
    ai_r = pickle.load(fr)

/content/med-deadend/lifegate


In [9]:
q_table = np.zeros((10, 10, 5))
q_d = np.zeros_like(q_table)
q_r = np.zeros_like(q_table)


for i in range(10):
    for j in range(10):
        for a in range(5):
            key = tuple([j, i, a])
            try:
                q_table[i,j,a] = ai.q[key]
                q_d[i,j,a] = ai_d.q[key]
                q_r[i,j,a] = ai_r.q[key]
            except:
                pass

In [10]:
import yaml
import random

In [11]:
from lifegate import LifeGate

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
params = yaml.safe_load(open('config.yaml', 'r'))

In [13]:
np.random.seed(seed=params['random_seed'])
random.seed(params['random_seed'])
random_state = np.random.RandomState(params['random_seed'])

In [14]:
env = LifeGate(max_steps=params['episode_max_len'], state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.1)

In [15]:
env_30 = LifeGate(max_steps=30, state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.1)

In [16]:
import Shaping

In [17]:
from Shaping import *

In [18]:
%cd /content/Shaping

from choose_actions import action_probs_top_n_epsilon
from shaping_features import *
from gen_policies import *
from IS import *
from subset_policies import *
from v_pi_e import *
from optimization import *
from neural_net import *
from prep_variance import *
from SCOPE_variance import SCOPE_variance

/content/Shaping


# Generating policies

## interm updates

In [ ]:
def experiment_actions(nb_episodes, env, action_probs):
    """
    Run the experiment for a specified number of episodes.

    Parameters:
    - nb_episodes: Number of episodes
    - env: Experiment environment
    - action_probs: Action probabilities

    Returns:
    - policies: List of policies (pi_b or pi_e)
    """
    # Define the dtype for the structured array
    dtype = [
        ('state_last', np.float64, (2,)),
        ('action', np.int64),
        ('reward', np.float64),
        ('state', np.float64, (2,)),
        ('timestep', np.int64),
        ('psi', np.float64)
    ]

    policies = []
    for i in range(nb_episodes):
        trajectory = np.empty(0, dtype=dtype)
        s = env.reset()
        env.render()
        term = False
        timestep = 0
        while not term:
            state_last = s
            print(s)
            action = choose_action(tuple(s), action_probs)
            s, r, term, _ = env.step(action)

            psi = smallest_distance_to_deadend(state_last, env)
            data_point = np.array([(state_last, action, r, s, timestep, psi)], dtype=dtype)
            trajectory = np.append(trajectory, data_point)
            timestep += 1

        policies.append(trajectory)

    with open('policies.pkl', 'wb') as f:
        pickle.dump(policies, f)

    return policies

In [ ]:
P_pi_b_good = np.zeros((10, 10, 5))

P_pi_b_good[2,9,3] = 1
P_pi_b_good[1,9,3] = 1
P_pi_b_good[0,9,1] = 1
P_pi_b_good[0,8,1] = 1
P_pi_b_good[0,7,1] = 1
P_pi_b_good[0,6,1] = 1
P_pi_b_good[0,5,1] = 1
P_pi_b_good[0,4,4] = 1
P_pi_b_good[1,4,4] = 1
P_pi_b_good[2,4,4] = 1
P_pi_b_good[3,4,4] = 1
P_pi_b_good[4,4,4] = 1
P_pi_b_good[5,4,1] = 1
P_pi_b_good[5,5,1] = 1
P_pi_b_good[5,4,1] = 1
P_pi_b_good[5,3,1] = 1
P_pi_b_good[5,2,1] = 1
P_pi_b_good[5,1,1] = 1

In [ ]:
P_pi_b_good_stoch = np.zeros((10, 10, 5))

P_pi_b_good_stoch[2,9,3] = 0.5
P_pi_b_good_stoch[2,9,1] = 0.5

P_pi_b_good_stoch[2,8,3] = 0.5
P_pi_b_good_stoch[2,8,1] = 0.5

P_pi_b_good_stoch[2,7,3] = 0.5
P_pi_b_good_stoch[2,7,1] = 0.5

P_pi_b_good_stoch[1,9,3] = 0.5
P_pi_b_good_stoch[1,9,1] = 0.5

P_pi_b_good_stoch[1,8,3] = 0.5
P_pi_b_good_stoch[1,8,1] = 0.5

P_pi_b_good_stoch[1,7,3] = 0.5
P_pi_b_good_stoch[1,7,1] = 0.5

P_pi_b_good_stoch[2,6,3] = 1
P_pi_b_good_stoch[1,6,3] = 1
P_pi_b_good_stoch[1,5,3] = 1


P_pi_b_good_stoch[0,9,1] = 1
P_pi_b_good_stoch[0,8,1] = 1
P_pi_b_good_stoch[0,7,1] = 1
P_pi_b_good_stoch[0,6,1] = 1
P_pi_b_good_stoch[0,5,1] = 1

P_pi_b_good_stoch[0,4,1] = 0.5
P_pi_b_good_stoch[0,4,4] = 0.5
P_pi_b_good_stoch[0,4,1] = 0.5
P_pi_b_good_stoch[0,4,4] = 0.5
P_pi_b_good_stoch[0,3,1] = 0.5
P_pi_b_good_stoch[0,3,4] = 0.5
P_pi_b_good_stoch[0,2,1] = 0.5
P_pi_b_good_stoch[0,2,4] = 0.5

P_pi_b_good_stoch[1,4,1] = 0.5
P_pi_b_good_stoch[1,4,4] = 0.5
P_pi_b_good_stoch[1,4,1] = 0.5
P_pi_b_good_stoch[1,4,4] = 0.5
P_pi_b_good_stoch[1,3,1] = 0.5
P_pi_b_good_stoch[1,3,4] = 0.5
P_pi_b_good_stoch[1,2,1] = 0.5
P_pi_b_good_stoch[1,2,4] = 0.5

P_pi_b_good_stoch[2,4,1] = 0.5
P_pi_b_good_stoch[2,4,4] = 0.5
P_pi_b_good_stoch[2,4,1] = 0.5
P_pi_b_good_stoch[2,4,4] = 0.5
P_pi_b_good_stoch[2,3,1] = 0.5
P_pi_b_good_stoch[2,3,4] = 0.5
P_pi_b_good_stoch[2,2,1] = 0.5
P_pi_b_good_stoch[2,2,4] = 0.5

P_pi_b_good_stoch[3,4,1] = 0.5
P_pi_b_good_stoch[3,4,4] = 0.5
P_pi_b_good_stoch[3,4,1] = 0.5
P_pi_b_good_stoch[3,4,4] = 0.5
P_pi_b_good_stoch[3,3,1] = 0.5
P_pi_b_good_stoch[3,3,4] = 0.5
P_pi_b_good_stoch[3,2,1] = 0.5
P_pi_b_good_stoch[3,2,4] = 0.5

P_pi_b_good_stoch[4,4,1] = 0.5
P_pi_b_good_stoch[4,4,4] = 0.5
P_pi_b_good_stoch[4,4,1] = 0.5
P_pi_b_good_stoch[4,4,4] = 0.5
P_pi_b_good_stoch[4,3,1] = 0.5
P_pi_b_good_stoch[4,3,4] = 0.5
P_pi_b_good_stoch[4,2,1] = 0.5
P_pi_b_good_stoch[4,2,4] = 0.5

P_pi_b_good_stoch[0,1,4] = 1
P_pi_b_good_stoch[1,1,4] = 1
P_pi_b_good_stoch[2,1,4] = 1
P_pi_b_good_stoch[3,1,4] = 1
P_pi_b_good_stoch[4,1,4] = 1

P_pi_b_good_stoch[5,4,1] = 1
P_pi_b_good_stoch[5,3,1] = 1
P_pi_b_good_stoch[5,2,1] = 1
P_pi_b_good_stoch[5,1,1] = 1



In [ ]:
P_pi_b_bad = np.zeros((10, 10, 5))

P_pi_b_bad[2,9,4] = 1

P_pi_b_bad[3,9,1] = 1
P_pi_b_bad[3,8,1] = 1
P_pi_b_bad[3,7,1] = 1
P_pi_b_bad[3,6,4] = 1

P_pi_b_bad[4,6,2] = 1
P_pi_b_bad[4,7,2] = 1
P_pi_b_bad[4,8,2] = 1
P_pi_b_bad[4,9,4] = 1

P_pi_b_bad[5,9,1] = 1
P_pi_b_bad[5,8,1] = 1
P_pi_b_bad[5,7,1] = 1
P_pi_b_bad[5,6,4] = 1

P_pi_b_bad[6,6,4] = 1
P_pi_b_bad[7,6,4] = 1
P_pi_b_bad[8,6,4] = 1






In [ ]:
env.main_deaths

[[9, 0],
 [9, 1],
 [9, 2],
 [9, 3],
 [9, 4],
 [9, 5],
 [9, 6],
 [9, 7],
 [9, 8],
 [9, 9],
 [8, 0]]

# Gen Policies

In [19]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env, P_pi_e)

In [20]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env_30, P_pi_b)

P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env_30, P_pi_e)

# Prep

In [19]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env, P_pi_e)

In [20]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [22]:
testing = SCOPE_variance(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [23]:
IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor, weight_first_tensor = testing.prepare()

/content/Shaping/SCOPE_variance.py:92: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = self.dtype)


In [32]:
model_1000 = train_var(model, 100, 0.001, padded_state_tensors, states_first_tensor, states_last_tensor, testing)

Epoch 1
Var loss:  tensor(0.0998, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 0.24141247356427334
E_s_wdiff_all_sq: 0.14142863628335298
E_IS_SCOPE: -0.0017230963821165332
E_IS_E_SCOPE: -0.0016126965713371732
Total Loss: 0.09976416278242806
----------------------------------------
Epoch 2
Var loss:  tensor(0.0848, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 0.2316856550412188
E_s_wdiff_all_sq: 0.14670070607329533
E_IS_SCOPE: -0.0016323729192394868
E_IS_E_SCOPE: -0.0015294829381487425
Total Loss: 0.08478029412880841
----------------------------------------
Epoch 3
Var loss:  tensor(0.0715, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 0.22324671679199845
E_s_wdiff_all_sq: 0.1515315713222341
E_IS_SCOPE: -0.001544206371644443
E_IS_E_SCOPE: -0.0014486263603

In [46]:
# Prepare input data
input_data = torch.tensor([0, 6], dtype=torch.float64)

# Pass input through the model
model(input_data)

tensor([0.1457], dtype=torch.float64, grad_fn=<ViewBackward0>)

In [21]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [22]:
test1 = SCOPE_variance(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [27]:
# IS_tensor, samples_IS, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor = test1.prepare()

# Modified class with bootstrap_all_terms
IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor = test1.prepare()

ValueError: too many values to unpack (expected 6)

In [ ]:
timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = test1.prep_policies()

In [ ]:
states_output, states_first_output, states_last_output = test1.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
sums_states_weight_diff = test1.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
gamma_weights_states_last_sub_states_first = test1.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)


# sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)
samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_all_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)


In [ ]:
sum_terms = sums_states_weight_diff + gamma_weights_states_last_sub_states_first
IS_SCOPE_1 = IS_tensor*sum_terms

In [ ]:
SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first
E_IS_SCOPE = torch.mean(torch.mean(samples_IS, dim =1) * torch.mean(SCOPE, dim =1))
E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

E_IS_SCOPE_2 = torch.mean(torch.mean(samples_IS_SCOPE, dim =1))
E_IS_E_SCOPE_2 = torch.mean(torch.mean(samples_IS,dim = 1 )) * torch.mean(torch.mean(samples_all_shaping, dim =1))

In [ ]:
E_IS_SCOPE

tensor(-1.1470e+24, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
E_IS_SCOPE_2

tensor(-1.5943e+26, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
samples_IS_SCOPE[0]

tensor([ -2.5405e-85,  -0.0000e+00,   7.9024e-62,  -2.9790e-61,  -3.1831e-75,
         -1.5365e-64,  -0.0000e+00,  -8.3521e-71,   9.1314e-24,  -6.9648e-66,
        -2.0605e-135,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,
         -1.5136e-54,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,
          0.0000e+00,   7.9024e-62,   0.0000e+00,  -0.0000e+00,  2.1164e-111,
        -9.5684e-135,  -0.0000e+00, -1.4140e-116,  -3.3219e-82,  -0.0000e+00,
         -1.5197e-87,  -2.5695e-67,   5.3077e-82,  -0.0000e+00,   2.3741e-47,
         -0.0000e+00,  -1.8253e-71,  -3.4879e+11,  -0.0000e+00,  -0.0000e+00,
         3.8146e-114,  -0.0000e+00,  -7.5975e-31,  1.2321e-124,  -0.0000e+00,
         -0.0000e+00, -2.0958e-170,   0.0000e+00,  -0.0000e+00,  -0.0000e+00,
         -0.0000e+00, -1.0845e-110,  -1.8398e-69,  -1.8253e-71,   0.0000e+00,
         -1.0142e-82,  -3.8087e-71, -4.7846e-116,  -0.0000e+00,  -1.1575e+14,
         -8.3983e-61,  -0.0000e+00,  -3.5652e-20,  -4.5806e-83, 

In [ ]:
sample_sums_states_weight_diff[0]

tensor([ 5.1683e-01,  5.2720e-01,  2.3982e+00,  5.2951e-01,  5.1357e-01,
         5.1364e-01, -1.1535e+00,  5.1631e-01,  5.1753e-01,  5.1325e-01,
         9.0952e-02,  3.6275e-01,  5.2720e-01,  7.6119e-02,  7.6752e-02,
         5.3759e-01,  9.0952e-02,  7.6337e-02,  9.0952e-02,  9.0657e-02,
         5.7605e+00,  2.3982e+00,  3.6205e+00,  9.0436e-02,  3.9866e+00,
         7.6361e-02,  5.1632e-01,  7.7653e-02,  1.2674e-01,  9.0436e-02,
         7.6119e-02,  5.1709e-01,  3.6920e+00,  5.1663e-01,  9.0973e-02,
         5.3759e-01,  5.3762e-01, -2.9977e+06,  5.1357e-01,  5.3602e-01,
         4.1946e+00,  5.2720e-01, -6.4696e+01,  5.2645e-01,  5.1683e-01,
         5.2907e-01,  5.2645e-01,  3.6920e+00,  5.3771e-01,  4.8009e-01,
         4.2201e-01,  9.0952e-02,  8.3283e-02,  5.3762e-01,  4.2676e+00,
         5.1366e-01,  7.7653e-02,  5.3583e-01,  5.3728e-01, -5.6362e+07,
         2.5242e-01, -1.5415e+10, -1.8276e-02,  8.9459e-02,  9.0952e-02,
         5.1665e-01, -2.1687e+00, -4.6237e+01,  5.2

In [ ]:
sums_shaping = sums_states_weight_diff + gamma_weights_states_last_sub_states_first

In [ ]:
torch.mean(sums_shaping)

tensor(-9.7904e+39, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
# Print the min and max values of tensors
print("padded_weight_diff_tensors - Min:", padded_weight_diff_tensors.min().item(), " Max:", padded_weight_diff_tensors.max().item())
print("sums_states_weight_diff - Min:", sums_states_weight_diff.min().item(), " Max:", sums_states_weight_diff.max().item())
print("gamma_weights_last_tensor - Min:", gamma_weights_last_tensor.min().item(), " Max:", gamma_weights_last_tensor.max().item())
print("samples_IS - Min:", samples_IS.min().item(), " Max:", samples_IS.max().item())


padded_weight_diff_tensors - Min: -6979572071212940.0  Max: 1.839130399958296e+16
sums_states_weight_diff - Min: -5225342891061462.0  Max: 1365487.6953395614
gamma_weights_last_tensor - Min: 1.846463839642936e-238  Max: 143894458319084.28
samples_IS - Min: -10076.89890923087  Max: 159882731465649.2


In [ ]:
torch.mean(sample_sums_states_weight_diff,dim=1)

tensor([-9.9641e+39, -2.6692e+27, -9.9641e+39,  ..., -1.9928e+40,
        -2.9892e+40, -1.3346e+27], dtype=torch.float64,
       grad_fn=<MeanBackward1>)

In [ ]:
torch.mean(samples_gamma_weight_states_last_sub_states_first, dim =1)

tensor([1.7368e+38, 5.5852e+13, 1.7368e+38,  ..., 3.4735e+38, 5.2103e+38,
        2.7925e+13], dtype=torch.float64, grad_fn=<MeanBackward1>)

# Calc Variance

In [31]:
def calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE):

  # states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)

  # Begin calcs without clamping

  # IS
  E_IS_sq = torch.mean(torch.mean(samples_IS, dim = 1)**2)
  E_IS_all_sq = torch.mean(torch.mean(samples_IS, dim = 1))**2

  # states_weight_diff
  E_s_wdiff_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim =1)**2)
  E_s_wdiff_all_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim = 1))**2

  # all terms
  SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first
  E_IS_SCOPE = torch.mean(torch.mean(samples_IS, dim =1) * torch.mean(SCOPE, dim =1))
  E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

  # E_IS_SCOPE = torch.mean(torch.mean(samples_IS_SCOPE, dim =1))
  # E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 )) * torch.mean(torch.mean(samples_all_shaping, dim =1))



  # SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_IS_E_SCOPE
  SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_s_wdiff_all_sq

  IS_variance = E_IS_sq - E_IS_all_sq

  return E_IS_sq, E_IS_all_sq, E_s_wdiff_sq, E_s_wdiff_all_sq, E_IS_SCOPE, E_IS_E_SCOPE, IS_variance, SCOPE_variance


In [ ]:
SCOPE = sample_sums_states_weight_diff-samples_gamma_weight_states_last_sub_states_first


In [ ]:
_,_,_,_,_,_,IS_variance, SCOPE_variance = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE)

In [ ]:
print(f'IS Variance: {IS_variance.item()} SCOPE Variance: {SCOPE_variance.item()}')

IS Variance: 2.514807940970177e+22 SCOPE Variance: 5.892057226414652e+25


In [ ]:
SCOPE_variance

tensor([2.0262e+82, 1.7582e+80, 2.0262e+82,  ..., 4.0349e+82, 6.0435e+82,
        1.7582e+80], dtype=torch.float64, grad_fn=<SubBackward0>)

# Optimizing

In [ ]:
import torch.optim as optim

In [29]:
import torch
import torch.optim as optim

def train_var(model, num_epochs, learning_rate, padded_state_tensors, states_first_tensor, states_last_tensor, test1):
    model.train()

    # Enable anomaly detection
    torch.autograd.set_detect_anomaly(True)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0

        # Forward pass
        states_output, states_first_output, states_last_output = test1.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
        sums_states_weight_diff = test1.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
        gamma_weights_states_last_sub_states_first = test1.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output, weight_first_tensor)
        # sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)

        samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_all_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)


        E_IS_sq, E_IS_all_sq, E_s_wdiff_sq, E_s_wdiff_all_sq, E_IS_SCOPE, E_IS_E_SCOPE, _, variance_loss = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE)

        print(f"Epoch {epoch+1}")
        print("Var loss: ", variance_loss)

        # Print each term
        print(f"E_IS_sq: {E_IS_sq}")
        print(f"E_IS_all_sq: {E_IS_all_sq}")
        print(f"E_s_wdiff_sq: {E_s_wdiff_sq}")
        print(f"E_s_wdiff_all_sq: {E_s_wdiff_all_sq}")
        print(f"E_IS_SCOPE: {E_IS_SCOPE}")
        print(f"E_IS_E_SCOPE: {E_IS_E_SCOPE}")

        tot = variance_loss

        optimizer.zero_grad()

        # Retain the graph to avoid clearing it before backward pass
        tot.backward(retain_graph=True)

        optimizer.step()

        total_loss += tot.item()

        print(f"Total Loss: {total_loss}")
        print("-" * 40)

    # Disable anomaly detection after running the code
    torch.autograd.set_detect_anomaly(False)

    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"Parameter name: {name}")
            print(f"Weights: {param.data}")

    return model


In [ ]:
model2 = train_var(model, 20, 0.0001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(-3.0838e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 5.210967549166126e+22
E_s_wdiff_all_sq: 2.5172984363039316e+22
E_IS_SCOPE: -1.5459488896936224e+25
E_IS_E_SCOPE: -1.4417566494849713e+22
Total Loss: -3.0838057890344423e+25
----------------------------------------
Epoch 2
Var loss:  tensor(-3.3585e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 6.546944000346388e+22
E_s_wdiff_all_sq: 3.517525168586692e+22
E_IS_SCOPE: -1.6839284068856547e+25
E_IS_E_SCOPE: -1.8983210591791954e+22
Total Loss: -3.358515944880221e+25
----------------------------------------
Epoch 3
Var loss:  tensor(-3.6329e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 8.127328468140865e+22
E_s_wdiff_all_sq: 4.669789629828819e+22
E_IS_SCOPE: -1.82179

In [ ]:
model5 = train_var(model4, 500, 0.0001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(1.6838e+21, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.2080531216272573e+22
E_s_wdiff_all_sq: 7.854446300022197e+18
E_IS_SCOPE: -3.2924307594909997e+22
E_IS_E_SCOPE: -1.0101281026055919e+22
Total Loss: 1.683838514322106e+21
----------------------------------------
Epoch 2
Var loss:  tensor(2.4018e+21, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.5681370975049804e+22
E_s_wdiff_all_sq: 5.3651102132735515e+20
E_IS_SCOPE: -2.958920317364135e+22
E_IS_E_SCOPE: -6.916923741107582e+21
Total Loss: 2.401815260791626e+21
----------------------------------------
Epoch 3
Var loss:  tensor(2.3126e+21, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.060980670643068e+22
E_s_wdiff_all_sq: 2.9444178650483524e+20
E_IS_SCOPE: -3.6706298454

KeyboardInterrupt: 

In [ ]:
model2 = train_var(model, 200, 0.0001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(-5.0872e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.2091641687126665e+22
E_s_wdiff_all_sq: 8.421209012866424e+18
E_IS_SCOPE: -2.547315218031649e+25
E_IS_E_SCOPE: -1.2421832478324517e+22
Total Loss: -5.087179914210118e+25
----------------------------------------
Epoch 2
Var loss:  tensor(-5.3797e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.1588018418409055e+22
E_s_wdiff_all_sq: 8.04536804186359e+20
E_IS_SCOPE: -2.694291367756086e+25
E_IS_E_SCOPE: -1.7349946687008094e+22
Total Loss: -5.379704141723259e+25
----------------------------------------
Epoch 3
Var loss:  tensor(-5.6711e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.4111873352395464e+22
E_s_wdiff_all_sq: 3.463374565893483e+21
E_IS_SCOPE: -2.8408122

KeyboardInterrupt: 

In [ ]:
model3 = train_var(model, 20, 0.001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(-1.3998e+26, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 2.7000520983736854e+26
E_s_wdiff_all_sq: 1.1991158006418987e+26
E_IS_SCOPE: -2.0238984353616566e+26
E_IS_E_SCOPE: 1.7432066784634587e+24
Total Loss: -1.3997894919094345e+26
----------------------------------------
Epoch 2
Var loss:  tensor(-1.8930e+26, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 2.4851380256092592e+26
E_s_wdiff_all_sq: 1.0868796875981285e+26
E_IS_SCOPE: -2.1642866384407017e+26
E_IS_E_SCOPE: 1.6592145943890643e+24
Total Loss: -1.892960208309719e+26
----------------------------------------
Epoch 3
Var loss:  tensor(-2.3750e+26, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 2.2864222596610805e+26
E_s_wdiff_all_sq: 9.831123864853387e+25
E_IS_SCOPE: -2.307

# Working on prep

In [ ]:
# # Working on padding trajectories to allow for easier optimization

# def pad_trajectories(pi_b):
#     # Find the maximum length among all trajectories
#     max_length = max(len(traj) for traj in pi_b)

#     # Define the padding value
#     padding_value = np.array([np.array([0, 0]), 0, 0, np.array([0, 0]), 0, 0], dtype=object)

#     # Pad each trajectory to match the maximum length
#     padded_pi_b = [traj + [padding_value] * (max_length - len(traj)) for traj in pi_b]

#     return padded_pi_b

In [ ]:
# pi_b_padded = pad_trajectories(pi_b)

In [ ]:
def variance_terms_tens(eval_policy, behav_policy, behavior_policies):
  # Initialize lists to store axis data for each policy
  timesteps = []
  states = []
  state_first = []
  state_last = []
  actions = []
  rewards = []
  gamma_last = []
  weight_last = []
  weights = calculate_importance_weights(eval_policy, behav_policy, behavior_policies)
  psi = []

  for index, policy in enumerate(behavior_policies):
      policy_array = np.array(policy)
      timesteps.append(policy_array[:, 4].astype(int))
      # s.append(policy_array[:, 0])

      # last timestep for gamma
      gamma_last.append(len(policy))
      # last importance weight
      weight_last.append(weights[index][-1])


      states.append(policy_array[:, 0][1:])
      psi.append(policy_array[:,5][1:])
      state_first.append(policy_array[:,0][0])
      state_last.append(policy_array[:,0][-1])
      actions.append(policy_array[:, 1])
      rewards.append(policy_array[:, 2].astype(float))

  weights_difference = []
  for index, weight in enumerate(weights):
    # diff = np.array(w[index][:-1]) - np.array(w[index][1:])
    diff = np.array(weight[:-1]) - np.array(weight[1:])
    weights_difference.append(diff)

  return timesteps, states, state_first, state_last, actions, rewards, gamma_last, weight_last, weights, weights_difference

In [ ]:
# t, s, s_f, s_l, a, r, g_l, w_l, w, w_diff = variance_terms_tens(P_pi_e, P_pi_b, pi_b)

In [ ]:
timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = variance_terms_tens(P_pi_e, P_pi_b, pi_b)

In [ ]:
import numpy as np

def padding_IS_terms(timesteps, actions, rewards, weights):
    # Find the maximum length among all lists
    max_length = max(len(traj) for traj in timesteps)

    # Define the padding values
    zero_padding = 0

    # Pad each list to match the maximum length
    padded_timesteps = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in timesteps]
    padded_rewards = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in rewards]
    padded_actions = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in actions]
    padded_weights = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights]

    return padded_timesteps, padded_rewards, padded_actions, padded_weights

In [ ]:
padded_timesteps, padded_rewards, padded_actions, padded_weights = padding_IS_terms(timesteps, actions, rewards, weights)

In [ ]:
def padding_states_weights_difference(states, weights_difference):
    # Find the maximum length of trajectories
    max_length = max(len(trajectory) for trajectory in states)

    zero_padding = 0

    # Pad each trajectory to make them all the same length
    padded_states = [
        [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
        for trajectory in states
    ]

    padded_weights_difference = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights_difference]

    return padded_states, padded_weights_difference

In [ ]:
padded_states, padded_weights_difference = padding_states_weights_difference(states, weights_difference)

In [ ]:
def tensorize_padded_terms(padded_states, padded_weights_difference):
  padded_state_tensors = torch.tensor(padded_states, dtype = torch.float64)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = torch.float64)
  padded_weight_diff_tensors = padded_weight_diff_tensors.unsqueeze(-1)

  return padded_state_tensors, padded_weight_diff_tensors


In [ ]:
padded_state_tensors, padded_weight_diff_tensors = tensorize_padded_terms(padded_states, padded_weights_difference)

<ipython-input-31-3c22136e8020>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = torch.float64)


In [ ]:
def tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last):
  states_first_tensor = torch.tensor(states_first, dtype = torch.float64)
  states_last_tensor = torch.tensor(states_last, dtype = torch.float64)
  gamma_last_tensor = torch.tensor(gamma_last, dtype = torch.float64)
  weights_last_tensor = torch.tensor(weights_last, dtype = torch.float64)

  return states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor

In [ ]:
states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)

In [ ]:
def calc_IS_terms(gamma, timesteps, rewards, weights):
  gtrw = np.power(gamma, timesteps)*rewards*weights

  IS_tensor = torch.sum(torch.tensor(gtrw, dtype = torch.float32), dim = 1, keepdim = True)

  return IS_tensor


In [ ]:
IS_tensor = calc_IS_terms(0.9, padded_timesteps, padded_rewards, padded_weights)

In [ ]:
torch.finfo(torch.float64).max

1.7976931348623157e+308

In [ ]:
# samples_IS[0]

In [ ]:
def calc_gamma_weight_last(gamma, gamma_last, weights_last):
  gamma_weight_last = np.power(gamma, gamma_last)*weights_last

  gamma_weight_last_tensor = torch.tensor(gamma_weight_last, dtype = torch.float64).unsqueeze(-1)

  return gamma_weight_last_tensor

In [ ]:
gamma_weights_last_tensor = calc_gamma_weight_last(0.9, gamma_last, weights_last)

In [ ]:
def bootstrap_IS_terms(IS_tensor, num_samples):
  seed = 42
  torch.manual_seed(seed)

  num_bootstraps = num_samples*len(IS_tensor)

  # Sample indices with replacement
  sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

  # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
  new_size = (num_samples, IS_tensor.shape[0])

  IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

  # sampled_tensor = IS_bootstraps.view(new_size)

  return IS_bootstraps

In [ ]:
samples_IS = bootstrap_IS_terms(IS_tensor, 10000)

In [ ]:
def clamp_large_terms(input_term):
  max_float64 = torch.finfo(torch.float64).max
  return torch.clamp(input_term, min=-max_float64, max=max_float64)


In [ ]:
def clamp_large_terms(input_term):
    max_float64 = torch.tensor(torch.finfo(torch.float64).max, dtype=torch.float64)
    min_value = torch.tensor(-1e50, dtype=torch.float64)
    return torch.clamp(input_term, min=min_value, max=max_float64)

    # return torch.clamp(input_term, min=-max_float64, max=max_float64)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CustomizableFeatureNet_d(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_prob=0.2, dtype=torch.float32):
        super(CustomizableFeatureNet_d, self).__init__()
        self.hidden_layers = nn.ModuleList()

        # Create the hidden layers based on the provided sizes
        for in_dim, out_dim in zip([input_dim] + hidden_dims, hidden_dims):
            self.hidden_layers.append(nn.Linear(in_dim, out_dim).to(dtype))

        self.output_layer = nn.Linear(hidden_dims[-1], output_dim).to(dtype)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [ ]:
model = CustomizableFeatureNet_d(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [ ]:
# model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
def pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor):
  # Get model outputs for states
  states_output = model(padded_state_tensors)
  states_first_output = model(states_first_tensor)
  states_last_output = model(states_last_tensor)
  return states_output, states_first_output, states_last_output

In [ ]:
states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)

In [ ]:
def states_weight_diff_sums(states_output, padded_weight_diff_tensors):
  states_weight_diff = states_output * padded_weight_diff_tensors
  sums_states_weight_diff = torch.sum(states_weight_diff, dim =1)

  return sums_states_weight_diff

In [ ]:
sums_states_weight_diff = states_weight_diff_sums(states_output, padded_weight_diff_tensors)

In [ ]:
def last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output):
  gamma_weights_states_last_sub_states_first = gamma_weights_last_tensor*states_last_output -  states_first_output

  return gamma_weights_states_last_sub_states_first

In [ ]:
gamma_weights_states_last_sub_states_first = last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)

In [ ]:
# gamma_weights_states_last_sub_states_first.shape

In [ ]:
def bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, num_samples):

  seed = 42
  torch.manual_seed(seed)

  num_bootstraps = num_samples*sums_states_weight_diff.shape[0]

  # Sample indices with replacement
  sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps,), dtype=torch.long)

  # sizes
  # size_states_weights_diff = (num_samples, states_output.shape[0], states_output.shape[1])
  reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

  # Resize samples to shape num_samples x num_trajectories x length_padded_trajectories
  # samples_states_output = states_output[sampled_indices].view(size_states_weights_diff)
  # samples_weight_diff = padded_weight_diff_tensors[sampled_indices].view(size_states_weights_diff)

  # Resize samples to shape num_samples x num_trajectories
  sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
  # samples_states_first_output = sums_states_weight_diff[sampled_indices].view(reshaped_size)
  # samples_states_last_output = states_last_output[sampled_indices].view(reshaped_size)
  samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)


  return sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first


In [ ]:
sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first = bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, 10000)

In [ ]:
# def gtrw_plus_states_weight_diff_min_last_firsts_terms(gtrw_tensor, sums_states_weight_diff, gamma_weights_states_last_sub_states_first):
#   sum_IS_and_shaping = gtrw_tensor + sums_states_weight_diff + gamma_weights_states_last_sub_states_first

#   return sum_IS_and_shaping

In [ ]:
# def bootstrap_IS_shaping_terms(gtrw_tensor, sums_states_weight_diff, gamma_weights_states_last_sub_states_first):
#   seed = 42
#   torch.manual_seed(seed)

#   num_bootstraps = num_samples*sums_states_weight_diff.shape[0]

#   # Sample indices with replacement
#   sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps,), dtype=torch.long)



In [ ]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim = 1))))

In [ ]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim =1)**2)))

In [ ]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim =1))**2) - torch.mean(clamp_large_terms(torch.mean(samples_IS, dim = 1))))

In [ ]:
# SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first

In [ ]:
# torch.mean(SCOPE,dim =1).shape

In [ ]:
torch.mean(clamp_large_terms(samples_IS), dim = 1)

tensor([      -inf, 2.0056e+14,       -inf,  ...,       -inf,       -inf,
        1.0028e+14])

In [ ]:
def calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first):


  # states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)



  # Begin calcs without clamping

  # IS
  E_IS_sq = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1)**2)
  E_IS_all_sq = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1))**2

  # states_weight_diff
  E_s_wdiff_sq = torch.mean(torch.mean(clamp_large_terms(sample_sums_states_weight_diff), dim =1)**2)
  E_s_wdiff_all_sq = torch.mean(torch.mean(clamp_large_terms(sample_sums_states_weight_diff), dim = 1))**2

  # all terms
  SCOPE = clamp_large_terms(sample_sums_states_weight_diff)+clamp_large_terms(samples_gamma_weight_states_last_sub_states_first)
  E_IS_SCOPE = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim =1) * torch.mean(SCOPE, dim =1))
  E_IS_E_SCOPE = torch.mean(torch.mean(clamp_large_terms(samples_IS),dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

  SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_IS_E_SCOPE

  IS_variance = E_IS_sq - E_IS_all_sq

  return IS_variance, SCOPE_variance


In [ ]:
IS_variance, SCOPE_variance = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first)

In [ ]:
torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1)**2)

tensor(inf)

In [ ]:
SCOPE_variance

tensor(nan, dtype=torch.float64, grad_fn=<SubBackward0>)

In [ ]:
IS_variance

tensor(nan)

In [ ]:
import torch

def clamp_large_terms(input_term):
    max_float64 = torch.tensor(torch.finfo(torch.float64).max, dtype=torch.float64)
    # min_value = torch.tensor(-1e38, dtype=torch.float64)
    min_value = torch.tensor(-9e-38, dtype=torch.float64)

    # min_value = torch.tensor(-1.7e+308, dtype = torch.float64)
    # min_value = torch.tensor(torch.finfo(torch.float64).min, dtype=torch.float64)


    # Using torch.where to explicitly set values outside the desired range
    clamped_result = torch.where(input_term < min_value, min_value, input_term)
    clamped_result = torch.where(clamped_result > max_float64, max_float64, clamped_result)

    return clamped_result
clamp_large_terms(samples_IS)[0].min()

tensor(-9.0000e-38)

In [ ]:
samples_IS[0]

tensor([ 0.0000e+00, -9.5567e-34, -1.1020e-12, -5.7703e-20,  0.0000e+00,
        -6.6609e-06,  0.0000e+00,  0.0000e+00,  0.0000e+00, -7.4562e-17,
        -1.3518e-33, -6.3068e-29, -1.1566e-08,  0.0000e+00, -3.2522e-24,
         0.0000e+00,  0.0000e+00, -4.0924e-02,  0.0000e+00,  0.0000e+00,
        -6.0833e-21, -1.1020e-12,  0.0000e+00, -3.5032e-43, -3.4013e-37,
        -7.9832e-23,  0.0000e+00, -2.2156e-12,  0.0000e+00, -7.3671e-27,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00, -3.0953e-12,  0.0000e+00, -7.1745e-11,  0.0000e+00,
        -1.2184e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -4.8370e-19,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -1.7865e-25, -3.0953e-12, -3.1972e-14,
         0.0000e+00, -3.9939e-21,  0.0000e+00, -4.2383e-17,  0.0000e+00,
         0.0000e+00, -9.5567e-34,  0.0000e+00,  0.0000e+00, -5.4262e-07,
        -3.7487e-37, -2.6165e-41,  0.0000e+00, -1.0

In [ ]:
clamp_large_terms(samples_IS)[0].min()

tensor(-9.0000e-38)

In [ ]:
def clamp_large_terms(tensor):
    max_float64 = torch.finfo(torch.float64).max
    min_float64 = torch.finfo(torch.float64).min

    tensor[tensor == float('inf')] = 1e38  # Choose a large finite value
    tensor[tensor == float('-inf')] = -1e38  # Choose a small finite value

    return tensor

# Example usage:
# samples_IS_clamped = clamp_tensor(samples_IS)

In [ ]:
# samples_IS_clamped[0]

NameError: name 'samples_IS_clamped' is not defined

In [ ]:
torch.tensor(torch.finfo(torch.float64).min, dtype=torch.float64)

tensor(-1.7977e+308, dtype=torch.float64)

In [ ]:
model.eval()

CustomizableFeatureNet_d(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=32, bias=True)
  )
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
output = model(state_tensors)

RuntimeError: mat1 and mat2 must have the same dtype, but got Float and Double

In [ ]:
output.shape

In [ ]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
scope_set, phi_set = subset_policies(pi_b, 0.3)

In [ ]:
IS, state_tensors, w_diff, f, st_og, psi_res, sample_last_tensors = variance_terms_tens(P_pi_e, P_pi_b, phi_set)

In [ ]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
model2 = train_mse_var(model, 40, 0.001, 1, 1, IS, state_tensors, w_diff, f, sample_last_tensors, phi_set, st_og, psi_res)

# Test class

In [19]:
# import numpy as np
# from IS import calculate_importance_weights

# import torch

class SCOPE_variance_play(object):
    def __init__(self, model, gamma, num_bootstraps, pi_b, P_pi_b, P_pi_e, dtype):
        self.model = model
        self.gamma = gamma
        self.num_bootstraps = num_bootstraps
        self.pi_b = pi_b
        self.P_pi_b = P_pi_b
        self.P_pi_e = P_pi_e
        self.dtype = dtype


    def prep_policies(self):
        # Initialize lists to store axis data for each policy
        timesteps = []
        states = []
        state_first = []
        state_last = []
        actions = []
        rewards = []
        gamma_last = []
        weight_last = []
        weight_first = []
        # all_weights_temp, weights = calculate_importance_weights(self.P_pi_e, self.P_pi_b, self.pi_b)
        weights, all_weights_temp = calculate_importance_weights(self.P_pi_e, self.P_pi_b, self.pi_b)
        psi = []

        for index, policy in enumerate(self.pi_b):
            policy_array = np.array(policy)
            timesteps.append(policy_array['timestep'].astype(int))
            # s.append(policy_array[:, 0])

            # last timestep for gamma
            gamma_last.append(len(policy))
            # last importance weight
            weight_last.append(weights[index][-1])
            weight_first.append(weights[index][0])


            states.append(policy_array['state'][1:])
            psi.append(policy_array['psi'][1:])
            state_first.append(policy_array['state'][0])
            state_last.append(policy_array['state'][-1])
            actions.append(policy_array['action'])
            rewards.append(policy_array['reward'].astype(float))

        weights_difference = []
        for index, weight in enumerate(weights):
            # diff = np.array(w[index][:-1]) - np.array(w[index][1:])
            diff = np.array(weight[:-1]) - np.array(weight[1:])
            weights_difference.append(diff)

        return timesteps, states, state_first, state_last, actions, rewards, gamma_last, weight_last, weights, weights_difference, weight_first

    def padding_IS_terms(self,timesteps, actions, rewards, weights):
        # Find the maximum length among all lists
        max_length = max(len(traj) for traj in timesteps)

        # Define the padding values
        zero_padding = 0

        # Pad each list to match the maximum length
        padded_timesteps = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in timesteps]
        padded_rewards = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in rewards]
        padded_actions = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in actions]
        padded_weights = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights]

        return padded_timesteps, padded_rewards, padded_actions, padded_weights

    def padding_states_weights_difference(self, states, weights_difference):
        # Find the maximum length of trajectories
        max_length = max(len(trajectory) for trajectory in states)

        zero_padding = 0

        # Pad each trajectory to make them all the same length
        padded_states = [
            [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
            for trajectory in states
        ]

        padded_weights_difference = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights_difference]

        return padded_states, padded_weights_difference

    def tensorize_padded_terms(self, padded_states, padded_weights_difference):
        padded_state_tensors = torch.tensor(padded_states, dtype = self.dtype)
        padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = self.dtype)
        padded_weight_diff_tensors = padded_weight_diff_tensors.unsqueeze(-1)

        return padded_state_tensors, padded_weight_diff_tensors

    def tensorize_last_and_first_terms(self, states_first, states_last, gamma_last, weights_last):
        states_first_tensor = torch.tensor(states_first, dtype = self.dtype)
        states_last_tensor = torch.tensor(states_last, dtype = self.dtype)
        gamma_last_tensor = torch.tensor(gamma_last, dtype = self.dtype)
        weights_last_tensor = torch.tensor(weights_last, dtype = self.dtype)

        return states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor

    def calc_IS_terms(self, gamma, timesteps, rewards, weights):
        gtrw = np.power(gamma, timesteps)*rewards*weights

        IS_tensor = torch.sum(torch.tensor(gtrw, dtype = self.dtype), dim = 1, keepdim = True)

        return IS_tensor

    def calc_gamma_weight_last(self, gamma, gamma_last, weights_last):
        gamma_weight_last = np.power(gamma, gamma_last)*weights_last

        gamma_weight_last_tensor = torch.tensor(gamma_weight_last, dtype = self.dtype).unsqueeze(-1)

        return gamma_weight_last_tensor

    def tensorize_weight_first(self, weights_first):

      weight_first_tensor = torch.tensor(weights_first, dtype = torch.float64).unsqueeze(-1)

      return weight_first_tensor
    def bootstrap_IS_terms(self, IS_tensor, num_samples):
        seed = 42
        torch.manual_seed(seed)

        num_bootstraps = num_samples*len(IS_tensor)

        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

        # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
        new_size = (num_samples, IS_tensor.shape[0])

        IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

        # sampled_tensor = IS_bootstraps.view(new_size)

        return IS_bootstraps

    def states_weight_diff_sums(self, states_output, padded_weight_diff_tensors):
        states_weight_diff = states_output * padded_weight_diff_tensors
        sums_states_weight_diff = torch.sum(states_weight_diff, dim =1)

        return sums_states_weight_diff

    def last_first_terms_operations(self, gamma_weights_last_tensor, states_last_output, states_first_output, weight_first_tensor):
        gamma_weights_states_last_sub_states_first = gamma_weights_last_tensor*states_last_output -  weight_first_tensor*states_first_output

        return gamma_weights_states_last_sub_states_first


    def bootstrap_shaping_terms(self, sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor):

        seed = 42
        torch.manual_seed(seed)

        num_samples = self.num_bootstraps

        num_bootstraps_lin = num_samples*sums_states_weight_diff.shape[0]

        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps_lin,), dtype=torch.long)

        reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

        # Resize samples to shape num_samples x num_trajectories
        sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
        samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)

        # Sum states_weight_diff and gamma_weights-states_last_sub_states_first
        sum_terms = sums_states_weight_diff + gamma_weights_states_last_sub_states_first

        # sample IS terms

        IS_SCOPE = IS_tensor * sum_terms

        samples_IS_SCOPE = IS_SCOPE[sampled_indices].view(reshaped_size)


        sample_all_shaping = sum_terms[sampled_indices].view(reshaped_size)

        return sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, sample_all_shaping, samples_IS_SCOPE

    def bootstrap_all_terms(self, sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor):
        seed = 42
        torch.manual_seed(seed)

        # num_bootstraps = num_samples*len(IS_tensor)

        # Sample indices with replacement
        # sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

        # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
        # new_size = (num_samples, IS_tensor.shape[0])

        # IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

        num_samples = self.num_bootstraps
        num_bootstraps_lin = num_samples*sums_states_weight_diff.shape[0]


        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps_lin,), dtype=torch.long)

        reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

        IS_bootstraps = IS_tensor[sampled_indices].view(reshaped_size)

        # Resize samples to shape num_samples x num_trajectories
        sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
        samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)

        # Sum states_weight_diff and gamma_weights-states_last_sub_states_first
        sum_terms = sums_states_weight_diff + gamma_weights_states_last_sub_states_first

        # sample IS terms

        IS_SCOPE = IS_tensor * sum_terms

        samples_IS_SCOPE = IS_SCOPE[sampled_indices].view(reshaped_size)

        sample_all_shaping = sum_terms[sampled_indices].view(reshaped_size)

        return IS_bootstraps, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, sample_all_shaping, samples_IS_SCOPE



    def pass_states(self,model, padded_state_tensors, states_first_tensor, states_last_tensor):
        # Get model outputs for states
        states_output = model(padded_state_tensors)
        states_first_output = model(states_first_tensor)
        states_last_output = model(states_last_tensor)
        return states_output, states_first_output, states_last_output


    # def prepare(self):
    #     timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = self.prep_policies()
    #     padded_timesteps, padded_rewards, padded_actions, padded_weights = self.padding_IS_terms(timesteps, actions, rewards, weights)
    #     padded_states, padded_weights_difference = self.padding_states_weights_difference(states, weights_difference)
    #     padded_state_tensors, padded_weight_diff_tensors = self.tensorize_padded_terms(padded_states, padded_weights_difference)
    #     states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = self.tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)
    #     IS_tensor = self.calc_IS_terms(self.gamma, padded_timesteps, padded_rewards, padded_weights)
    #     gamma_weights_last_tensor = self.calc_gamma_weight_last(self.gamma, gamma_last, weights_last)
    #     samples_IS = self.bootstrap_IS_terms(IS_tensor, self.num_bootstraps)

    #     return IS_tensor, samples_IS, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor


    def prepare(self):
        timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference, weight_first = self.prep_policies()
        padded_timesteps, padded_rewards, padded_actions, padded_weights = self.padding_IS_terms(timesteps, actions, rewards, weights)
        padded_states, padded_weights_difference = self.padding_states_weights_difference(states, weights_difference)
        padded_state_tensors, padded_weight_diff_tensors = self.tensorize_padded_terms(padded_states, padded_weights_difference)
        states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = self.tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)
        IS_tensor = self.calc_IS_terms(self.gamma, padded_timesteps, padded_rewards, padded_weights)
        gamma_weights_last_tensor = self.calc_gamma_weight_last(self.gamma, gamma_last, weights_last)
        weight_first_tensor = self.tensorize_weight_first(weight_first)
        # samples_IS = self.bootstrap_IS_terms(IS_tensor, self.num_bootstraps)

        return IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor, weight_first_tensor

In [20]:
def calculate_importance_weights(eval_policy, behav_policy, behavior_policies):
    """
    Calculate importance weights for behavior policies.

    Parameters:
    - eval_policy: Evaluation policy
    - behav_policy: Behavior policy
    - behavior_policies: List of behavior policies

    Returns:
    - all_weights: List of importance weights
    """
    all_weights_temp = []
    for trajectory in behavior_policies:
        cum_ratio = 1
        cumul_weights = []
        for step in trajectory:
            # eval_action_probs = get_quadrant_policy(step[0], eval_policy)
            # behav_action_probs = get_quadrant_policy(step[0], behav_policy)

            P_pi_b = behav_policy[tuple(np.append(step[0].astype(int) , (step[1],)))]
            P_pi_e = eval_policy[tuple(np.append(step[0].astype(int) , (step[1],)))]

            # ratio = (0.8*eval_action_probs[step[1]] +0.2*0.25)/ (0.8*behav_action_probs[step[1]]+0.2*0.25)
            ratio = P_pi_e/P_pi_b
            cum_ratio *= ratio
            cumul_weights.append(cum_ratio)
        all_weights_temp.append(cumul_weights)

        all_weights = [list(np.cumprod(i)) for i in all_weights_temp]

    return all_weights_temp, all_weights

# Test variance

In [21]:
def calculate_shaped_variance_play(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE):

  # states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)

  # Begin calcs without clamping

  # IS
  E_IS_sq = torch.mean(torch.mean(samples_IS, dim = 1)**2)
  E_IS_all_sq = torch.mean(torch.mean(samples_IS, dim = 1))**2

  # states_weight_diff
  E_s_wdiff_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim =1)**2)
  E_s_wdiff_all_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim = 1))**2

  # all terms
  SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first
  E_IS_SCOPE = torch.mean(torch.mean(samples_IS, dim =1) * torch.mean(SCOPE, dim =1))
  E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

  # samples_IS_SCOPE seems to be wrong, get means and then multiply
  # E_IS_SCOPE = torch.mean(torch.mean(samples_IS_SCOPE, dim =1))
  # E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 )) * torch.mean(torch.mean(samples_all_shaping, dim =1))



  # SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_IS_E_SCOPE
  SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_s_wdiff_all_sq

  IS_variance = E_IS_sq - E_IS_all_sq

  return E_IS_sq, E_IS_all_sq, E_s_wdiff_sq, E_s_wdiff_all_sq, E_IS_SCOPE, E_IS_E_SCOPE, IS_variance, SCOPE_variance


# Optimizing play

In [22]:
import torch
import torch.optim as optim

def train_var_play(model, num_epochs, learning_rate, padded_state_tensors, states_first_tensor, states_last_tensor, test1):
    model.train()

    # Enable anomaly detection
    torch.autograd.set_detect_anomaly(True)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0

        # Forward pass
        states_output, states_first_output, states_last_output = test1.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
        sums_states_weight_diff = test1.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
        gamma_weights_states_last_sub_states_first = test1.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output, weight_first_tensor)
        # sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)

        samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_all_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)


        E_IS_sq, E_IS_all_sq, E_s_wdiff_sq, E_s_wdiff_all_sq, E_IS_SCOPE, E_IS_E_SCOPE, _, variance_loss = calculate_shaped_variance_play(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE)

        print(f"Epoch {epoch+1}")
        print("Var loss: ", variance_loss)

        # Print each term
        print(f"E_IS_sq: {E_IS_sq}")
        print(f"E_IS_all_sq: {E_IS_all_sq}")
        print(f"E_s_wdiff_sq: {E_s_wdiff_sq}")
        print(f"E_s_wdiff_all_sq: {E_s_wdiff_all_sq}")
        print(f"E_IS_SCOPE: {E_IS_SCOPE}")
        print(f"E_IS_E_SCOPE: {E_IS_E_SCOPE}")

        tot = variance_loss

        optimizer.zero_grad()

        # Retain the graph to avoid clearing it before backward pass
        tot.backward(retain_graph=True)

        optimizer.step()

        total_loss += tot.item()

        print(f"Total Loss: {total_loss}")
        print("-" * 40)

    # Disable anomaly detection after running the code
    torch.autograd.set_detect_anomaly(False)

    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"Parameter name: {name}")
            print(f"Weights: {param.data}")

    return model


In [27]:
states_output, states_first_output, states_last_output = testing.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
sums_states_weight_diff = testing.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
gamma_weights_states_last_sub_states_first = testing.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output, weight_first_tensor)


# sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = testing.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)
samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = testing.bootstrap_all_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)


In [28]:
sum_SCOPE = sums_states_weight_diff + gamma_weights_states_last_sub_states_first

seed = 42
torch.manual_seed(seed)

# num_bootstraps = num_samples*len(IS_tensor)

# Sample indices with replacement
# sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

# new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
# new_size = (num_samples, IS_tensor.shape[0])

# IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

num_samples = 10000
num_bootstraps_lin = num_samples*sums_states_weight_diff.shape[0]


# Sample indices with replacement
sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps_lin,), dtype=torch.long)

reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

# IS_bootstraps = IS_tensor[sampled_indices].view(reshaped_size)

# # Resize samples to shape num_samples x num_trajectories
# sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
# samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)

# Sum states_weight_diff and gamma_weights-states_last_sub_states_first
sum_terms = sums_states_weight_diff + gamma_weights_states_last_sub_states_first


samples_sum_terms = sum_terms[sampled_indices].view(reshaped_size)
# sample IS terms

IS_SCOPE = IS_tensor * sum_terms

samples_IS_SCOPE = IS_SCOPE[sampled_indices].view(reshaped_size)

sample_all_shaping = sum_terms[sampled_indices].view(reshaped_size)



In [ ]:
sample_sums_states_weight_diff + samples_gamma_weight_states_last_sub_states_first

In [31]:
SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first
E_IS_SCOPE = torch.mean(torch.mean(samples_IS, dim =1) * torch.mean(SCOPE, dim =1))
E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

# I believe this is wrong, need to get means for full samples first then expected value
E_IS_SCOPE_2 = torch.mean(torch.mean(samples_IS_SCOPE, dim =1))
E_IS_E_SCOPE_2 = torch.mean(torch.mean(samples_IS,dim = 1 )) * torch.mean(torch.mean(samples_all_shaping, dim =1))

In [34]:
samples_IS[0].shape

torch.Size([1000])

In [32]:
SCOPE.shape

torch.Size([10000, 1000])

In [68]:
torch.mean(torch.mean(samples_IS*SCOPE, dim = 1))

tensor(0.1475, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [51]:
torch.mean(torch.mean(IS_tensor*sum_terms))

tensor(0.1482, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [50]:
torch.mean(torch.mean(samples_IS, dim =1))

tensor(0.0044, dtype=torch.float64)

In [48]:
torch.mean(torch.mean(samples_IS_SCOPE, dim =1))

tensor(0.1475, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [47]:
torch.mean(torch.mean(samples_sum_terms, dim =1))

tensor(0.3184, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
torch.mean

In [38]:
torch.mean(torch.mean(SCOPE, dim =1))

tensor(0.3184, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [39]:
torch.mean(torch.mean(samples_all_shaping, dim =1))

tensor(0.3184, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [65]:
E_IS_SCOPE

tensor(0.0015, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [36]:
E_IS_SCOPE_2

tensor(0.1475, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [35]:
E_IS_E_SCOPE

tensor(0.0014, dtype=torch.float64, grad_fn=<MulBackward0>)

In [37]:
E_IS_E_SCOPE_2

tensor(0.0014, dtype=torch.float64, grad_fn=<MulBackward0>)

# Class play

In [23]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env, P_pi_e)

In [24]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [25]:
testing = SCOPE_variance_play(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [26]:
IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor, weight_first_tensor = testing.prepare()

<ipython-input-19-41662d94a8d9>:92: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = self.dtype)


In [27]:
model_1000 = train_var_play(model, 100, 0.001, padded_state_tensors, states_first_tensor, states_last_tensor, testing)

Epoch 1
Var loss:  tensor(0.4364, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 0.91612813590345
E_s_wdiff_all_sq: 0.47977377573226215
E_IS_SCOPE: 0.0018630194424027102
E_IS_E_SCOPE: 0.0018434664174459066
Total Loss: 0.43639459134416786
----------------------------------------
Epoch 2
Var loss:  tensor(0.0953, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 0.10377027388910455
E_s_wdiff_all_sq: 0.00848972906796978
E_IS_SCOPE: -0.0002716328863720476
E_IS_E_SCOPE: -0.00026625847027983576
Total Loss: 0.09527092111201678
----------------------------------------
Epoch 3
Var loss:  tensor(0.0876, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 0.09273451330953343
E_s_wdiff_all_sq: 0.005075734755404952
E_IS_SCOPE: -0.00023341896140770455
E_IS_E_SCOPE: -0.000227844925

In [48]:
# P_pi_b[tuple(np.append(pi_b[0]['state'][0].astype(int) , (pi_b[0]['action'][0],)))]

In [ ]:
# padded_timesteps, padded_rewards, padded_actions, padded_weights = testing.padding_IS_terms(timesteps, actions, rewards, weights)

In [ ]:
# padded_states, padded_weights_difference = testing.padding_states_weights_difference(states, weights_difference)
# padded_state_tensors, padded_weight_diff_tensors = testing.tensorize_padded_terms(padded_states, padded_weights_difference)
# states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = testing.tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)
# IS_tensor = testing.calc_IS_terms(testing.gamma, padded_timesteps, padded_rewards, padded_weights)

In [39]:
# states_output, states_first_output, states_last_output = testing.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
# sums_states_weight_diff = testing.states_weight_diff_sums(states_output, padded_weight_diff_tensors)

# gamma_weights_last_tensor = testing.calc_gamma_weight_last(testing.gamma, gamma_last, weights_last)

# gamma_weights_states_last_sub_states_first = testing.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output, weight_first_tensor)


# Testing

## 200 trajectories

200 Trajectories:

In [55]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(200, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(200, env, P_pi_e)

In [56]:
model_200 = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [57]:
testing = SCOPE_variance_play(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [58]:
IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor, weight_first_tensor = testing.prepare()

In [63]:
# single cumprod
# 200 trajectories
model_200_1 = train_var_play(model_200, 10000, 0.001, padded_state_tensors, states_first_tensor, states_last_tensor, testing)

Epoch 1
Var loss:  tensor(0.3342, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 6.232169163204178e-06
E_IS_all_sq: 4.491946079694819e-06
E_s_wdiff_sq: 0.19240302141744486
E_s_wdiff_all_sq: 0.018279117028370654
E_IS_SCOPE: 0.08042899655323252
E_IS_E_SCOPE: 0.00040847554319232933
Total Loss: 0.3341666866322381
----------------------------------------
Epoch 2
Var loss:  tensor(0.3110, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 6.232169163204178e-06
E_IS_all_sq: 4.491946079694819e-06
E_s_wdiff_sq: 0.18031519309864652
E_s_wdiff_all_sq: 0.02094448154389672
E_IS_SCOPE: 0.07617173294757792
E_IS_E_SCOPE: 0.0003613200212326285
Total Loss: 0.3109932776305239
----------------------------------------
Epoch 3
Var loss:  tensor(0.2886, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 6.232169163204178e-06
E_IS_all_sq: 4.491946079694819e-06
E_s_wdiff_sq: 0.1686902209108127
E_s_wdiff_all_sq: 0.02330613342299742
E_IS_SCOPE: 0.0719275724152696
E_IS_E_SCOPE: 0.0003163842112025963
T

KeyboardInterrupt: 

## 400 Trajectories

400 Trajectories:

In [38]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(400, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(400, env, P_pi_e)

In [39]:
model_400 = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [44]:
testing_400 = SCOPE_variance_play(model_400, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [45]:
IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor, weight_first_tensor = testing_400.prepare()

In [48]:
model_400 = train_var_play(model_400, 800, 0.0007, padded_state_tensors, states_first_tensor, states_last_tensor, testing_400)

Streaming output truncated to the last 5000 lines.
E_s_wdiff_sq: 3.818623501816317e-06
E_s_wdiff_all_sq: 5.1504677041227486e-08
E_IS_SCOPE: -1.5581479430685247e-06
E_IS_E_SCOPE: 4.658730291707227e-08
Total Loss: 1.9661124687824543e-06
----------------------------------------
Epoch 318
Var loss:  tensor(1.9614e-06, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.5443728380345596e-06
E_IS_all_sq: 2.135908702056001e-06
E_s_wdiff_sq: 3.820825995560044e-06
E_s_wdiff_all_sq: 5.349843717599916e-08
E_IS_SCOPE: -1.5618117588208178e-06
E_IS_E_SCOPE: 4.537144321640203e-08
Total Loss: 1.9614252902881636e-06
----------------------------------------
Epoch 319
Var loss:  tensor(1.9567e-06, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.5443728380345596e-06
E_IS_all_sq: 2.135908702056001e-06
E_s_wdiff_sq: 3.817179748592323e-06
E_s_wdiff_all_sq: 5.362001030560776e-08
E_IS_SCOPE: -1.5574684865952953e-06
E_IS_E_SCOPE: 5.019030585161591e-08
Total Loss: 1.9567062893714516e-06
-------------

1000 Trajectories:

In [45]:
# single cumprod
# 1000 trajectories
model11 = train_var_play(model10, 500, 0.0001, padded_state_tensors, states_first_tensor, states_last_tensor, 1, 1, testing)

Streaming output truncated to the last 5000 lines.
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 1.8435369920087568e-06
E_s_wdiff_all_sq: 9.069579199805096e-08
E_IS_SCOPE: -1.1038198776012357e-05
E_IS_E_SCOPE: -9.750861916403596e-06
Total Loss: 3.032905472266013e-07
----------------------------------------
Epoch 12
Var loss:  tensor(2.8471e-07, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 1.6681551818117882e-06
E_s_wdiff_all_sq: 1.4820307511116906e-10
E_IS_SCOPE: -1.013852417975717e-05
E_IS_E_SCOPE: -8.884314617475877e-06
Total Loss: 2.8471092060750724e-07
----------------------------------------
Epoch 13
Var loss:  tensor(3.2981e-07, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 1.7046211317725032e-06
E_s_wdiff_all_sq: 4.237948817006082e-08
E_IS_SCOPE: -9.460547193621885e-06
E_IS_E_SCOPE: -8.231770336608416e-06
Total Loss: 3.29

In [46]:
model12 = train_var_play(model11, 500, 0.0001, padded_state_tensors, states_first_tensor, states_last_tensor, 1, 1, testing)

Epoch 1
Var loss:  tensor(2.2263e-09, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 2.0454397267940005e-06
E_s_wdiff_all_sq: 5.467628114465457e-07
E_IS_SCOPE: -1.0254970373924974e-05
E_IS_E_SCOPE: -8.944183529765734e-06
Total Loss: 2.2262934623888673e-09
----------------------------------------
Epoch 2
Var loss:  tensor(6.5306e-08, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 1.65793187982493e-06
E_s_wdiff_all_sq: 1.2877612576471057e-07
E_IS_SCOPE: -9.075641433495659e-06
E_IS_E_SCOPE: -7.781154824562122e-06
Total Loss: 6.530560262656123e-08
----------------------------------------
Epoch 3
Var loss:  tensor(1.9184e-07, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 7.78221979305549e-06
E_IS_all_sq: 6.657096726622074e-06
E_s_wdiff_sq: 3.6306633195734738e-06
E_s_wdiff_all_sq: 1.7726246188113302e-06
E_IS_SCOPE: -1.18904501536152

KeyboardInterrupt: 

Greater policy randomness in pi_b

In [69]:
P_pi_b = action_probs_top_n_epsilon(q_table, 2, 0.4)
pi_b = experiment_actions(200, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 1, 0.05)
pi_e = experiment_actions(200, env, P_pi_e)
model_200_2_p4 = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)
testing = SCOPE_variance_play(model_200_2_p4, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)
IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor, weight_first_tensor = testing.prepare()


In [74]:
timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference, weights_first = testing.prep_policies()

Note that the variance within trajectories is quite low, which keeps the variance quite low, might need to consider normalizing in some manner, found this function of PDIS that does this based on maximum and minimum possible discounted reward

https://github.com/hari-sikchi/safeRL/blob/master/importance_sampling/importance_sampling.py



In [ ]:
# '''
# Per Decision Importance Sampling
# reward: list of reward obtained per time step

# gamma: discount factor
# trajectory_reward_high: Maximum value of sum of discounted rewards in a trajectory
# trajectory_reward_low: Minimum value of sum of discounted rewards in a trajectory

# returns normalized estimate of reward under evaluation policy
# '''

# def per_decision_is(pi_b,pi_e,gamma,reward,trajectory_reward_high,trajectory_reward_low):
#     horizon = len(reward)
#     expected_reward = 0
#     gamma_t = 1
#     importance_weight = 1
#     for t in range(1,horizon+1):
#         importance_weight *= pi_e[t-1]/pi_b[t-1]
#         expected_reward+= gamma_t * reward[t-1] *importance_weight
#         gamma_t *= gamma

#     return (expected_reward - trajectory_reward_low)/(trajectory_reward_high-trajectory_reward_low)

In [97]:
trajectory_reward_high = 0.9**16 * 1
trajectory_reward_low = 0.9**7 * -1

In [106]:
0.9**80

0.00021847450052839255

In [101]:
trajectory_reward_high

0.18530201888518416

In [102]:
trajectory_reward_low

-0.4782969000000001

In [99]:
(trajectory_reward_high-trajectory_reward_low)

0.6635989188851843

In [112]:
cum_ratio = 1
cumul_weights = []
P_pi_e_probs = []
P_pi_b_probs = []

for step in pi_b[0]:
    P_pi_b_prob = P_pi_b[tuple(np.append(step[0].astype(int), (step[1],)))]
    P_pi_e_prob = P_pi_e[tuple(np.append(step[0].astype(int), (step[1],)))]

    P_pi_b_probs.append(P_pi_b_prob)
    P_pi_e_probs.append(P_pi_e_prob)

    ratio = P_pi_e_prob / P_pi_b_prob
    cum_ratio *= ratio
    cumul_weights.append(cum_ratio)


In [114]:
P_pi_b_probs

[0.38,
 0.08,
 0.38,
 0.38,
 0.38,
 0.08,
 0.38,
 0.08,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.08,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.08,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.08,
 0.08,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.38,
 0.08,
 0.38,
 0.38]

In [118]:
P_pi_e[2,8,0]

0.01

In [116]:
pi_b[0]

array([([2., 9.], 1, 0., [2., 8.],  0, 3.),
       ([2., 8.], 0, 0., [2., 8.],  1, 3.),
       ([2., 8.], 1, 0., [2., 7.],  2, 3.),
       ([2., 7.], 1, 0., [2., 6.],  3, 3.),
       ([2., 6.], 3, 0., [1., 6.],  4, 3.),
       ([1., 6.], 2, 0., [1., 7.],  5, 4.),
       ([1., 7.], 3, 0., [0., 7.],  6, 4.),
       ([0., 7.], 0, 0., [0., 7.],  7, 5.),
       ([0., 7.], 1, 0., [0., 6.],  8, 5.),
       ([0., 6.], 3, 0., [0., 6.],  9, 5.),
       ([0., 6.], 1, 0., [0., 5.], 10, 5.),
       ([0., 5.], 1, 0., [0., 4.], 11, 5.),
       ([0., 4.], 1, 0., [0., 3.], 12, 6.),
       ([0., 3.], 2, 0., [0., 4.], 13, 7.),
       ([0., 4.], 1, 0., [0., 3.], 14, 6.),
       ([0., 3.], 4, 0., [1., 3.], 15, 7.),
       ([1., 3.], 0, 0., [1., 3.], 16, 6.),
       ([1., 3.], 2, 0., [1., 4.], 17, 6.),
       ([1., 4.], 0, 0., [1., 4.], 18, 5.),
       ([1., 4.], 0, 0., [1., 4.], 19, 5.),
       ([1., 4.], 0, 0., [1., 4.], 20, 5.),
       ([1., 4.], 0, 0., [1., 4.], 21, 5.),
       ([1., 4.], 1, 0., [1., 3.

In [115]:
P_pi_e_probs

[0.01,
 0.01,
 0.01,
 0.01,
 0.96,
 0.01,
 0.96,
 0.01,
 0.96,
 0.01,
 0.96,
 0.96,
 0.01,
 0.96,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.96,
 0.01,
 0.01,
 0.01,
 0.01,
 0.96,
 0.96,
 0.96,
 0.96,
 0.01,
 0.01,
 0.01,
 0.96,
 0.01,
 0.01,
 0.01,
 0.96,
 0.01,
 0.01,
 0.01,
 0.96]

In [108]:
pi_b[0]

array([([2., 9.], 1, 0., [2., 8.],  0, 3.),
       ([2., 8.], 0, 0., [2., 8.],  1, 3.),
       ([2., 8.], 1, 0., [2., 7.],  2, 3.),
       ([2., 7.], 1, 0., [2., 6.],  3, 3.),
       ([2., 6.], 3, 0., [1., 6.],  4, 3.),
       ([1., 6.], 2, 0., [1., 7.],  5, 4.),
       ([1., 7.], 3, 0., [0., 7.],  6, 4.),
       ([0., 7.], 0, 0., [0., 7.],  7, 5.),
       ([0., 7.], 1, 0., [0., 6.],  8, 5.),
       ([0., 6.], 3, 0., [0., 6.],  9, 5.),
       ([0., 6.], 1, 0., [0., 5.], 10, 5.),
       ([0., 5.], 1, 0., [0., 4.], 11, 5.),
       ([0., 4.], 1, 0., [0., 3.], 12, 6.),
       ([0., 3.], 2, 0., [0., 4.], 13, 7.),
       ([0., 4.], 1, 0., [0., 3.], 14, 6.),
       ([0., 3.], 4, 0., [1., 3.], 15, 7.),
       ([1., 3.], 0, 0., [1., 3.], 16, 6.),
       ([1., 3.], 2, 0., [1., 4.], 17, 6.),
       ([1., 4.], 0, 0., [1., 4.], 18, 5.),
       ([1., 4.], 0, 0., [1., 4.], 19, 5.),
       ([1., 4.], 0, 0., [1., 4.], 20, 5.),
       ([1., 4.], 0, 0., [1., 4.], 21, 5.),
       ([1., 4.], 1, 0., [1., 3.

In [107]:
weights[0]

[0.02631578947368421,
 0.003289473684210526,
 8.656509695290857e-05,
 2.2780288671818044e-06,
 5.755020296038243e-06,
 7.193775370047803e-07,
 1.817374830327866e-06,
 2.2717185379098326e-07,
 5.739078411561682e-07,
 1.510283792516232e-08,
 3.815453791619955e-08,
 9.639041157776727e-08,
 2.5365897783622966e-09,
 6.408226808494222e-09,
 1.686375475919532e-10,
 4.437830199788242e-12,
 1.167850052575853e-13,
 1.4598125657198164e-14,
 3.841612015052148e-16,
 1.010950530276881e-17,
 2.6603961323075813e-19,
 7.001042453441003e-21,
 1.7686844092903583e-20,
 4.654432656027258e-22,
 1.2248506989545416e-23,
 1.531063373693177e-24,
 4.029114141297834e-26,
 1.0178814672752422e-25,
 2.571490022590085e-25,
 6.496395846543372e-25,
 1.6411947401793782e-24,
 2.0514934252242227e-25,
 2.5643667815302784e-26,
 6.748333635605995e-28,
 1.7048421816267776e-27,
 4.4864267937546775e-29,
 1.1806386299354413e-30,
 3.1069437629880036e-32,
 7.849121085443377e-32,
 2.065558180379836e-33,
 2.581947725474795e-34,
 6.7

In [94]:
IS_tensor

tensor([[ 2.0551e-37],
        [-4.8452e-12],
        [ 3.4739e-26],
        [ 1.9805e-52],
        [ 5.6599e-30],
        [ 3.6186e-28],
        [ 1.2260e-37],
        [ 3.3817e-61],
        [ 4.8221e-73],
        [ 8.0587e-33],
        [ 2.3787e-34],
        [ 2.5989e-18],
        [ 1.1778e-45],
        [ 1.5114e-51],
        [ 1.0731e-23],
        [ 3.1264e-31],
        [ 4.8743e-37],
        [ 6.9782e-22],
        [ 7.5361e-54],
        [ 6.3329e-16],
        [ 2.5512e-42],
        [ 2.8200e-22],
        [ 1.0058e-30],
        [ 3.5798e-20],
        [ 4.5442e-51],
        [ 2.2281e-20],
        [ 1.8043e-30],
        [ 1.2700e-46],
        [ 1.1456e-37],
        [ 6.1262e-48],
        [ 2.1988e-26],
        [ 9.2430e-31],
        [ 7.5947e-22],
        [ 1.7820e-57],
        [ 5.8006e-42],
        [ 1.3517e-53],
        [ 3.3349e-24],
        [ 2.6211e-40],
        [ 3.5798e-20],
        [ 4.9731e-44],
        [ 1.4578e-21],
        [ 1.0805e-47],
        [ 4.2534e-27],
        [ 5

In [93]:
torch.var(IS_tensor)

tensor(3.8937e-07, dtype=torch.float64)

In [92]:
torch.mean(IS_tensor**2) - torch.mean(IS_tensor)**2

tensor(3.8742e-07, dtype=torch.float64)

In [91]:
torch.mean(IS_tensor)

tensor(4.4241e-05, dtype=torch.float64)

In [78]:
len(weights[0])

43

In [80]:
len(weights_difference[0])

42

In [77]:
len(pi_b[0])

43

In [70]:
# single cumprod
# 200 trajectories
model_200_2_p4 = train_var_play(model_200_2_p4, 200, 0.001, padded_state_tensors, states_first_tensor, states_last_tensor, 1, 1, testing)

Epoch 1
Var loss:  tensor(1.1985, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.826645974121427e-09
E_IS_all_sq: 1.9263712212949065e-09
E_s_wdiff_sq: 2.297909386211681
E_s_wdiff_all_sq: 1.0994508765271211
E_IS_SCOPE: 3.878983758204295e-05
E_IS_E_SCOPE: 3.843762575160876e-05
Total Loss: 1.1984592160084957
----------------------------------------
Epoch 2
Var loss:  tensor(0.0416, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.826645974121427e-09
E_IS_all_sq: 1.9263712212949065e-09
E_s_wdiff_sq: 0.04392587686827187
E_s_wdiff_all_sq: 0.0023080316182087575
E_IS_SCOPE: -8.007193464295585e-06
E_IS_E_SCOPE: -7.634354580536492e-06
Total Loss: 0.04161710147257035
----------------------------------------
Epoch 3
Var loss:  tensor(0.0388, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.826645974121427e-09
E_IS_all_sq: 1.9263712212949065e-09
E_s_wdiff_sq: 0.03925886301233597
E_s_wdiff_all_sq: 0.0004488434884700709
E_IS_SCOPE: -7.451422098909373e-06
E_IS_E_SCOPE: -7.078667

In [198]:
# doubly cumprod, not sure which is right for stepIS
model5 = train_var_play(model4, 50, 0.00001, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, testing)

Epoch 1
Var loss:  tensor(3.8975e+86, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.9050883499768664e+80
E_IS_all_sq: 2.6488687194959097e+80
E_s_wdiff_sq: 7.837653119776905e+86
E_s_wdiff_all_sq: 3.936778943589322e+86
E_IS_SCOPE: -4.922151070341052e+83
E_IS_E_SCOPE: -3.229592269140938e+83
Total Loss: 3.897490314804813e+86
----------------------------------------
Epoch 2
Var loss:  tensor(6.8576e+88, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.9050883499768664e+80
E_IS_all_sq: 2.6488687194959097e+80
E_s_wdiff_sq: 1.3783057143777487e+89
E_s_wdiff_all_sq: 6.925931966846359e+88
E_IS_SCOPE: 6.527757697009708e+84
E_IS_E_SCOPE: 4.283174687928771e+84
Total Loss: 6.857574106095142e+88
----------------------------------------
Epoch 3
Var loss:  tensor(4.5003e+87, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 3.9050883499768664e+80
E_IS_all_sq: 2.6488687194959097e+80
E_s_wdiff_sq: 9.044137064483694e+87
E_s_wdiff_all_sq: 4.545021682025533e+87
E_IS_SCOPE: 1.672171354213

# IS Play

In [125]:
def calculate_importance_weights(eval_policy, behav_policy, behavior_policies):
    """
    Calculate importance weights for behavior policies.

    Parameters:
    - eval_policy: Evaluation policy
    - behav_policy: Behavior policy
    - behavior_policies: List of behavior policies

    Returns:
    - all_weights: List of importance weights
    """
    all_weights_temp = []
    for trajectory in behavior_policies:
        cum_ratio = 1
        cumul_weights = []
        for step in trajectory:
            # eval_action_probs = get_quadrant_policy(step[0], eval_policy)
            # behav_action_probs = get_quadrant_policy(step[0], behav_policy)

            P_pi_b = behav_policy[tuple(np.append(step[0].astype(int) , (step[1],)))]
            P_pi_e = eval_policy[tuple(np.append(step[0].astype(int) , (step[1],)))]

            # ratio = (0.8*eval_action_probs[step[1]] +0.2*0.25)/ (0.8*behav_action_probs[step[1]]+0.2*0.25)
            ratio = P_pi_e/P_pi_b
            cum_ratio *= ratio
            cumul_weights.append(cum_ratio)
        all_weights_temp.append(cumul_weights)

        all_weights = [list(np.cumprod(i)) for i in all_weights_temp]

    return all_weights_temp, all_weights

In [126]:
all_weights_temp, all_weights = calculate_importance_weights(P_pi_e, P_pi_b, pi_b)

In [128]:
all_weights_temp[0]

[0.7132352941176471,
 0.5087045847750865,
 0.36282606414105434,
 0.2587803545711932,
 0.18457128230445397,
 0.1316427528200885,
 0.09389225752609254,
 0.06696727191199248,
 0.0477634218784064,
 0.03406655825151045,
 0.004258319781438806,
 0.0005322899726798508,
 0.00037964799522018767,
 0.0002707783495320456,
 3.38472936915057e-05,
 2.4141084471147448e-05,
 1.721827348309781e-05,
 2.1522841853872264e-06,
 1.304822287391006e-05,
 9.306453079185852e-06,
 1.1633066348982315e-06,
 8.297113498906504e-07,
 5.917794186720081e-07]

In [151]:
all_weights[0]

[0.7132352941176471,
 0.36282606414105434,
 0.13164275282008847,
 0.034066558251510434,
 0.006287708340180658,
 0.0008277312348312113,
 7.771755426316267e-05,
 5.204532588676245e-06,
 2.48586285712858e-07,
 8.468479182763697e-09,
 3.6061492422665387e-11,
 1.9195170816455204e-14,
 7.287408118376271e-18,
 1.973272342660357e-21,
 6.678992851535058e-26,
 1.612381306110978e-30,
 2.776242228765327e-35,
 5.9752622437758e-41,
 7.796655348684653e-46,
 7.255920717711713e-51,
 8.440860713209574e-57,
 7.003477936596074e-63,
 4.1445141020010597e-69]

In [106]:
# Extract probability for each state-action pair
num_steps, num_states, num_actions = P_pi_b.shape
num_samples = states_test.shape[0]

# Convert states_test to integers for indexing
states_int = states_test.astype(int)

# Extract probabilities using array indexing
probs_pi_b = P_pi_b[states_int[:, 0], states_int[:, 1], actions_test]
probs_pi_e = P_pi_e[states_int[:, 0], states_int[:, 1], actions_test]

In [107]:
probs_pi_b

array([0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.08,
       0.08, 0.68, 0.68, 0.08, 0.68, 0.68, 0.08, 0.08, 0.68, 0.08, 0.68,
       0.68])

In [108]:
probs_pi_e

array([0.485, 0.485, 0.485, 0.485, 0.485, 0.485, 0.485, 0.485, 0.485,
       0.485, 0.01 , 0.01 , 0.485, 0.485, 0.01 , 0.485, 0.485, 0.01 ,
       0.485, 0.485, 0.01 , 0.485, 0.485])

In [110]:
(probs_pi_e/probs_pi_b).cumprod(axis=-1)

array([7.13235294e-01, 5.08704585e-01, 3.62826064e-01, 2.58780355e-01,
       1.84571282e-01, 1.31642753e-01, 9.38922575e-02, 6.69672719e-02,
       4.77634219e-02, 3.40665583e-02, 4.25831978e-03, 5.32289973e-04,
       3.79647995e-04, 2.70778350e-04, 3.38472937e-05, 2.41410845e-05,
       1.72182735e-05, 2.15228419e-06, 1.30482229e-05, 9.30645308e-06,
       1.16330663e-06, 8.29711350e-07, 5.91779419e-07])

In [114]:
(probs_pi_e/probs_pi_b).prod(axis=-1)

5.917794186720081e-07

In [118]:
weights[0]

[0.7132352941176471,
 0.36282606414105434,
 0.13164275282008847,
 0.034066558251510434,
 0.006287708340180658,
 0.0008277312348312113,
 7.771755426316267e-05,
 5.204532588676245e-06,
 2.48586285712858e-07,
 8.468479182763697e-09,
 3.6061492422665387e-11,
 1.9195170816455204e-14,
 7.287408118376271e-18,
 1.973272342660357e-21,
 6.678992851535058e-26,
 1.612381306110978e-30,
 2.776242228765327e-35,
 5.9752622437758e-41,
 7.796655348684653e-46,
 7.255920717711713e-51,
 8.440860713209574e-57,
 7.003477936596074e-63,
 4.1445141020010597e-69]

In [122]:
weights

[[0.7132352941176471,
  0.36282606414105434,
  0.13164275282008847,
  0.034066558251510434,
  0.006287708340180658,
  0.0008277312348312113,
  7.771755426316267e-05,
  5.204532588676245e-06,
  2.48586285712858e-07,
  8.468479182763697e-09,
  3.6061492422665387e-11,
  1.9195170816455204e-14,
  7.287408118376271e-18,
  1.973272342660357e-21,
  6.678992851535058e-26,
  1.612381306110978e-30,
  2.776242228765327e-35,
  5.9752622437758e-41,
  7.796655348684653e-46,
  7.255920717711713e-51,
  8.440860713209574e-57,
  7.003477936596074e-63,
  4.1445141020010597e-69],
 [6.0625,
  26.214183134191178,
  80.84510557563685,
  177.82956323277526,
  278.98895939887166,
  2653.5166914163347,
  18000.702591032885,
  87094.33652773884,
  300554.4756778875,
  739757.3859855295,
  227596.42361250217,
  49942.971995499654,
  1369.9141292252755,
  26.80063802941003,
  0.37396395766511065,
  0.03163487428795419,
  0.001908689588734961,
  8.213671358375234e-05,
  2.1428466437824626e-05,
  3.9872887390228824e

In [123]:
weights_first = [trajectory[0] for trajectory in weights]


In [124]:
weights_first

[0.7132352941176471, 6.0625, 0.7132352941176471, 0.125, 0.7132352941176471]

In [95]:
states[0].astype(int)[:,0]

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 7, 8, 7])